In [28]:
import numpy as np
import pandas as pd
import pickle
import datetime
pd.set_option('display.max_rows',200)

In [2]:
data=pickle.load(open('data.pkl','rb'))

In [3]:
samples=pd.DataFrame(columns=['name','date','minute','data'])
for i,sample in data.iterrows():
    sel_name=data[data['name']==sample['name']]
    sel_date=sel_name[sel_name['date']==sample['date']]
    if sel_date.empty:
        samples=samples.append(sample,ignore_index=True)
    else:
        sel_date.sort_values(['minute'])
        last_minute=sel_date['minute'].iloc[-1]
        if sample['minute']-last_minute>8:
            samples=samples.append(sample,ignore_index=True)
        elif last_minute<80:
            samples=samples.append(sample,ignore_index=True)

In [6]:
result=[]
for i, row in samples.iterrows():
    sample=pd.Series()
    name,date,minute=row[['name','date','minute']]
    row=row['data']
    sample['name']=name
    sample['date']=date
    sample['league']=row['league']
    sample['minute']=row['minute']
    for c in ['name','corner','yellow','red','throw','freekick','goal']:
        sample[c+'_h'],sample[c+'_a']=row[c][0],row[c][1]
    if 'titles' in row:
        for i,c in enumerate(row['titles']):
            sample[c+'_h'],sample[c+'_a']=row['stats'][0,i],row['stats'][1,i]
    for c in ['Fulltime Result','Double Chance','Half Time Result','1st Goal']:
        sample[c+'_h']=row['odds'][c][0] if c in row['odds'].keys() else np.NaN
        sample[c+'_d']=row['odds'][c][1] if c in row['odds'].keys() else np.NaN
        sample[c+'_a']=row['odds'][c][2] if c in row['odds'].keys() else np.NaN
    result.append(sample)
result=pd.concat(result,axis=1).T


In [7]:
#find label
result['label']=np.NaN
for i, row in result.iterrows():
    sel=samples[samples['name']==name]
    sel=sel[sel['date']==date]
    sel=sel.sort_values('minute')
    sel_last=sel.iloc[-1]
    if sel_last['minute']>85:
        if sel_last['goal_h']>sel_last['goal_a']:
            result.iloc[i]['label']=1
        elif sel_last['goal_h']==sel_last['goal_a']:
            result.iloc[i]['label']=0
        elif sel_last['goal_h']<sel_last['goal_a']:
            result.iloc[i]['label']=-1

    sel_first=sel.iloc[0]
    if sel_last['minute']<5 and sel['goal_h']==0 and sel['goal_a']==0:
        sample['orig_full_result_h']=sel['Fulltime Result_h']
        sample['orig_full_result_d']=sel['Fulltime Result_d']
        sample['orig_full_result_a']=sel['Fulltime Result_a']

In [41]:
result[(result['date']==datetime.date(2018,9,27)) & (result['minute']>60)][['name','minute']]

,name,minute
412,Gubbio Albinoleffe,62
416,Sudtirol Fano,64
417,Vicenza Rimini,62
430,FC Nottingen Pforzheim,61
441,Slavia Prague FK Usti nad Labem,62
442,Aarhus Fremad AGF Aarhus,64
444,FC Roskilde AaB,67
453,Feyenoord U19 FC Utrecht U19,64
454,NAC U19 ADO Den Haag U19,62
455,Sparta Rotterdam U19 AZ U19,67


In [34]:
result[result['date']==datetime.date(2018,9,28)][['name','minute','goal_h','goal_a']].sort_values(['name','minute'])

,name,minute,goal_h,goal_a
926,Al Kuwait SC Ismaily SC,-1,2,3
885,Al Kuwait SC Ismaily SC,0,0,0
886,Al Kuwait SC Ismaily SC,2,0,0
887,Al Kuwait SC Ismaily SC,5,0,0
888,Al Kuwait SC Ismaily SC,7,0,0
889,Al Kuwait SC Ismaily SC,10,0,0
890,Al Kuwait SC Ismaily SC,12,0,0
891,Al Kuwait SC Ismaily SC,15,0,0
892,Al Kuwait SC Ismaily SC,18,0,0
893,Al Kuwait SC Ismaily SC,21,0,0


In [4]:
a

,name


In [36]:
row.keys()

dict_keys(['league', 'minute', 'name', 'corner', 'yellow', 'red', 'throw', 'freekick', 'goalkick', 'penalty', 'goal', 'titles', 'stats', 'odds'])

In [13]:
data.iloc[0]['data']

{'league': 'Italy Serie C',
 'minute': 6,
 'name': ['Pro Piacenza', 'Pistoiese'],
 'corner': [0, 0],
 'yellow': [0, 0],
 'red': [0, 0],
 'throw': [nan, nan],
 'freekick': [nan, nan],
 'goalkick': [nan, nan],
 'penalty': [0, 0],
 'goal': [0, 0],
 'titles': ['Attacks',
  'Dangerous Attacks',
  'Possession %',
  'On Target',
  'Off Target'],
 'stats': array([[ 7,  2, 38,  0,  0],
        [10,  5, 62,  0,  1]]),
 'odds': {'Fulltime Result': [2.0, 3.2, 3.75],
  'Double Chance': [1.25, 1.72, 1.33],
  'Half Time Result': [2.75, 1.9, 4.5],
  '1st Goal': [1.66, 6.5, 2.62],
  'Match Goals':      Over  Under
  2.5  2.37   1.53,
  'Alternative Match Goals':       Over  Under
  0.5   1.10  6.500
  1.5   1.44  2.620
  3.5   4.33  1.200
  4.5   8.00  1.062
  5.5  12.00  1.015,
  'First Half Goals':       Over  Under
  0.5   1.57   2.25
  1.5   3.75   1.25
  2.5  11.00   1.05}}